In [1]:
import numpy as np

In [2]:
sigma_y = np.array([[0,-1j],[1j,0]])
Omega_2 = np.kron(1j*sigma_y,1j*sigma_y)
Omega_3 = np.kron(1j*sigma_y,np.kron(1j*sigma_y,1j*sigma_y))

In [3]:
eigenvalues, eigenvectors = np.linalg.eig(1j*Omega_2)
W_neg = min(eigenvalues)

In [4]:
def pauli_y():
    return np.array([[0, -1j], [1j, 0]])

def i_sigma_y():
    return 1j * pauli_y()

def direct_sum_sigma_y(n):
    """
    Compute the direct sum of n copies of iσ_y
    """
    sigma_y_blocks = [i_sigma_y() for _ in range(n)]
    return np.block([[sigma_y_blocks[i] if i == j else np.zeros((2, 2)) for j in range(n)] for i in range(n)])

#for two parties
def Two_party_negativity(M):
  Pi_j = np.diag([1,-1,1,1])
  Ci_j = Pi_j*M*Pi_j.T
  sigma_y = np.array([[0,-1j],[1j,0]])
  Omega_2 = direct_sum_sigma_y(2)
  eigenvalues, eigenvectors = np.linalg.eig(1j*Omega_2*Ci_j)
  return min(eigenvalues)

def mu_negative_tri(K):
  sigma_y = np.array([[0,-1j],[1j,0]])
  Omega_3 = direct_sum_sigma_y(3)
  eigenvalues, eigenvectors = np.linalg.eig(1j*Omega_3*K)
  return min(eigenvalues)


#for three parties
def Three_party_negativity(M):
  P1_23 = np.diag([1, -1, 1, 1, 1, 1])
  P2_13 = np.diag([1, 1, 1, -1, 1, 1])
  P3_12 = np.diag([1, 1, 1, 1, 1, -1])

  C1_23 = mu_negative_tri(P1_23*M*P1_23.T)
  C2_13 = mu_negative_tri(P2_13*M*P2_13.T)
  C3_12 = mu_negative_tri(P3_12*M*P3_12)

  C2_3 = Two_party_negativity(C[2:6,2:6])   # Tracing out subsystem 1
  C1_3 = Two_party_negativity(np.kron(C[0:2,0:2], C[4:6,4:6])) # Tracing out subsystem 2
  C1_2 = Two_party_negativity(C[0:4,0:4])  # Tracing out subsystem 3

  tau1_23 = C1_23 - C1_2 - C1_3
  tau2_13 = C2_13 - C1_2 - C2_3
  tau3_12 = C3_12 - C1_3 - C2_3

  return min(tau1_23,tau2_13,tau3_12), C1_23, C2_13, C3_12